## Kein Mischportfolio, nur der Hebelanteil

In [168]:
import pandas as pd

# CSV mit historischen Daten einlesen
df = pd.read_csv("data/S&P 500 TR Historical Data.csv", delimiter=",", parse_dates=["Date"], dayfirst=False)
# Für die CSV mit Formaten wie "8,789.52" alle Kommas entfernen und zu Float konvertieren
for col in ["Price", "Open", "High", "Low"]:
    df[col] = df[col].str.replace(",", "").astype(float)
df = df.iloc[::-1].reset_index(drop=True)

# Indikatoren berechnen (analog zur Basisvariante, hier auf Price angewendet)
df["SMA_200"] = df["Price"].rolling(window=200).mean()
df["EMA_200"] = df["Price"].ewm(span=200, adjust=False).mean()
df["SellLine"] = df["SMA_200"] * 0.99
df["BuyLine"]  = df["SMA_200"] * 0.99

# Tagesrendite anhand von Price berechnen
df["Return"] = df["Price"].pct_change()

# Strategievariablen initialisieren
position = False                # True: in 3x gehebelter Position, False: in Cash
triggered_buy = False           # Markiert, dass ein Kaufsignal (durch Unterschreiten der BuyLine) lag
strategy_returns = [0]
portfolio_value = [100]         # Startkapital
tax_rate = 0.05             # Kapitalertragssteuersatz von 20%
last_entry_value = None         # Hier wird der Portfoliowert beim letzten Einstieg (Wechsel von Cash zu 3x lev) gespeichert

# Simulationsschleife: Iteriere über alle Tage
for i in range(1, len(df)):
    prev_close = df.iloc[i - 1]["Price"]
    today_close = df.iloc[i]["Price"]
    open_ = df.iloc[i]["Open"]
    high = df.iloc[i]["High"]
    low = df.iloc[i]["Low"]
    today_ret = df.iloc[i]["Return"]
    buy_line = df.iloc[i]["BuyLine"]
    sell_line = df.iloc[i]["SellLine"]

    day_return = 0
    exit_trade = False  # Kennzeichnet, ob heute ein Exit (3x lev -> Cash) stattfindet

    # === STRATEGIELOGIK ===
    if position:
        # Exit-Signal: Wir befinden uns in der gehebelten Position und der heutige Schlusskurs liegt unter der SellLine.
        if today_close < sell_line:
            if (sell_line - today_close) / sell_line > 0.005:
                # Es wird ein Exit-Preis berechnet (simulierte Ausstiegsorder)
                exit_price = sell_line * 0.995
                # Korrektur: Return berechnen wir ab dem vorherigen Schlusskurs bis zum Exit-Preis,
                # da wir zu diesem Preis verkauft haben (nicht bis today's close).
                ret = (exit_price - prev_close) / prev_close
            else:
                ret = (today_close - prev_close) / prev_close
            day_return = 3 * ret
            position = False        # Wechsel in Cash
            triggered_buy = (today_close < buy_line)
            exit_trade = True       # Wir haben einen Exit vollzogen, womit ggf. die Steuerabrechnung folgt.
        else:
            # Fortführung in der 3x gehebelten Position: Return wird gehebelter Tagesrendite berechnet.
            day_return = 3 * today_ret
    else:
        # Aktuell befinden wir uns in Cash.
        if triggered_buy:
            if today_close > buy_line:
                if (today_close - buy_line) / buy_line > 0.005:
                    entry_price = buy_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                position = True      # Einstieg in die 3x gehebelte Position
                triggered_buy = False
            else:
                day_return = 0
        else:
            if today_close > sell_line:
                if (today_close - sell_line) / sell_line > 0.005:
                    entry_price = sell_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                position = True
            elif today_close < buy_line:
                triggered_buy = True
                day_return = 0
            else:
                day_return = 0

    # === Intraday-Verletzungsabschlag ===
    apply_penalty = False
    if position:
        if open_ > sell_line and today_close > sell_line:
            if low < sell_line * 0.995:
                apply_penalty = True
    else:
        if open_ < buy_line and today_close < buy_line:
            if high > buy_line * 1.005:
                apply_penalty = True

    # Portfolio-Wert aktualisieren
    new_value = portfolio_value[-1] * (1 + day_return)
    if apply_penalty:
        new_value *= 0.985  # Risikokorrektur analog zum Basiscode

    # === Steuerliche Anpassung bei Exit (Wechsel von 3x lev zu Cash) ===
    if exit_trade and (last_entry_value is not None):
        profit = new_value - last_entry_value
        if profit > 0:
            tax_payment = tax_rate * profit
            new_value -= tax_payment
        # Referenzwert zurücksetzen, nachdem die Steuer abgezogen wurde.
        last_entry_value = None

    # === Beim Einstieg in eine 3x gehebelte Position: Referenzwert setzen ===
    if position and last_entry_value is None:
        last_entry_value = new_value

    strategy_returns.append(day_return)
    portfolio_value.append(new_value)

# Ergebnisse in DataFrame eintragen und anzeigen
df["Strategy Return"] = strategy_returns
df["Portfolio Value"] = portfolio_value

print(df[["Date", "Price", "Portfolio Value"]].head(1))
print(df[["Date", "Price", "Portfolio Value"]].tail(1))


        Date    Price  Portfolio Value
0 2001-08-13  1670.96            100.0
           Date    Price  Portfolio Value
4998 2021-06-21  8789.52     14385.506397


## Mischportfolio zwischen hebel und non Lev

In [169]:
import pandas as pd

# CSV mit historischen Daten einlesen und vorbereiten
df = pd.read_csv("data/S&P 500 TR Historical Data.csv", delimiter=",", parse_dates=["Date"], dayfirst=False)
# Für die CSV mit Formaten wie "8,789.52": Entferne Tausender-Trennzeichen und konvertiere zu float
for col in ["Price", "Open", "High", "Low"]:
    df[col] = df[col].str.replace(",", "").astype(float)
# Umkehrung der Reihenfolge, sodass das älteste Datum zuerst kommt
df = df.iloc[::-1].reset_index(drop=True)

# Indikatoren berechnen (hier basierend auf Price)
df["SMA_200"] = df["Price"].rolling(window=200).mean()
df["EMA_200"] = df["Price"].ewm(span=200, adjust=False).mean()
# Hier werden SellLine und BuyLine identisch definiert – du kannst sie bei Bedarf anpassen.
df["SellLine"] = df["SMA_200"] * 0.99
df["BuyLine"]  = df["SMA_200"] * 0.99

# Tagesrendite anhand von Price berechnen
df["Return"] = df["Price"].pct_change()

# --- SIMULATIONSEINSTELLUNGEN ---

# Startkapital, das später auf zwei Portfolio-Teile aufgeteilt wird:
initial_capital = 100

# Fixer Teil: 35 % des Portfolios, der täglich um 0.0004 wächst
fixed_return = 0.0004

# Simulierter Teil: 65 % des Portfolios – hier läuft die 3x leveraged Strategie
# (Steuer auf Gewinne wird beim Exit in dieser Komponente angewandt)
tax_rate = 0.05  # In deinem Code ist tax_rate = 0.05 (das entspricht 5%); passe dies bei Bedarf an.

# Initial allokierte Werte (bei Tradingereignis wird neu allokiert auf 35/65)
fixed_value = initial_capital * 0.35
sim_value = initial_capital * 0.65

# Für die Simulation des strategischen Teils bleiben die bisherigen Variablen bestehen:
position = False          # True, wenn der simulierte Teil aktuell in einer 3x gehebelten Position ist.
triggered_buy = False     # Signal, wenn Price unter die BuyLine gefallen ist.
last_entry_value = None   # Referenzwert beim letzten Einstieg in die simulierte Position (wichtig für die Steuerberechnung).

# Listen zum Tracking der Tagesergebnisse
total_history = [fixed_value + sim_value]
fixed_history = [fixed_value]
sim_history = [sim_value]

# Wir behalten den ursprünglichen Simulationscode (mit day_return) bei und erweitern diesen um das Rebalancing.
# Dabei setzen wir zusätzlich einen Flag "trade_event" (True bei Kauf oder Verkauf im simulierten Teil),
# der signalisiert, dass nach Anwendung des täglichen Returns eine Reallokation (35/65) erfolgen soll.

for i in range(1, len(df)):
    prev_close = df.iloc[i - 1]["Price"]
    today_close = df.iloc[i]["Price"]
    open_ = df.iloc[i]["Open"]
    high = df.iloc[i]["High"]
    low = df.iloc[i]["Low"]
    today_ret = df.iloc[i]["Return"]
    buy_line = df.iloc[i]["BuyLine"]
    sell_line = df.iloc[i]["SellLine"]

    day_return = 0
    exit_trade = False    # Kennzeichnet einen Exit (Verkauf) im simulierten Teil.
    trade_event = False   # Kennzeichnet, dass heute ein Trade (Einstieg oder Ausstieg) durchgeführt wurde.

    # === SIMULIERTE STRATEGIELOGIK (3x leveraged) ===
    if position:
        # Wir befinden uns in der 3x Position, schauen ob ein Exit-Signal vorliegt:
        if today_close < sell_line:
            if (sell_line - today_close) / sell_line > 0.005:
                # Starker Rückgang: Wir simulieren den Ausstieg zu einem Exit-Preis
                exit_price = sell_line * 0.995
                # Wichtiger Hinweis: Der Return wird dabei vom Vortagsschlusskurs bis zum Exitpreis berechnet,
                # da wir zu diesem Preis verkaufen.
                ret = (exit_price - prev_close) / prev_close
            else:
                ret = (today_close - prev_close) / prev_close
            day_return = 3 * ret  # 3x gehebelter Tagesreturn
            position = False     # Wir wechseln in Cash im simulierten Teil
            trade_event = True   # Es liegt ein Tradeereignis (Exit) vor.
            # Beim Exit prüfen wir, ob gleich ein Buy-Trigger (wegen möglicher weiterer Tiefstkurse) gesetzt werden soll:
            triggered_buy = (today_close < buy_line)

            # === Steuerliche Anpassung beim Exit im simulierten Teil ===
            if last_entry_value is not None:
                profit = sim_value * (1 + day_return) - last_entry_value
                if profit > 0:
                    tax_payment = tax_rate * profit
                    # Die Steuer wird nur auf den Gewinn angewandt.
                    # Hier reduzieren wir den simulierten Teil um den Steuerbetrag.
                    sim_value = sim_value * (1 + day_return) - tax_payment
                else:
                    sim_value = sim_value * (1 + day_return)
                last_entry_value = None
                # Wir setzen hier day_return für den simulierten Teil bereits in sim_value ein,
                # weshalb weiter unten beim Update des sim_value nicht nochmal multipliziert wird.
                # Deshalb setzen wir day_return = 0, da sim_value schon angepasst ist.
                day_return = 0
            else:
                # Sollte last_entry_value nicht gesetzt sein, einfach den Return anwenden:
                sim_value = sim_value * (1 + day_return)
        else:
            # Kein Exit-Signal: Wir verbleiben in der 3x Position, der simulierte Teil erhält den gehebelten Tagesreturn.
            day_return = 3 * today_ret
            sim_value = sim_value * (1 + day_return)
    else:
        # Wir sind aktuell im simulierten Cash – prüfen, ob ein Einstieg erfolgen soll:
        if triggered_buy:
            # Bereits ein Kauf-Trigger vorangegangen: Wir steigen ein, sobald der Kurs wieder über der BuyLine liegt.
            if today_close > buy_line:
                if (today_close - buy_line) / buy_line > 0.005:
                    entry_price = buy_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                position = True      # Eintritt in die 3x Position
                trade_event = True   # Es liegt ein Tradeereignis (Einstieg) vor.
                triggered_buy = False
                # Setze den Referenzwert für spätere Steuerberechnungen:
                last_entry_value = sim_value * (1 + day_return)
                sim_value = sim_value * (1 + day_return)
            else:
                # Kein Einstieg; simuliert wird in Cash (sim_value bleibt unverändert)
                day_return = 0
                sim_value = sim_value  # Keine Änderung
        else:
            # Direktes Einstiegssignal, falls der Schlusskurs über der SellLine liegt:
            if today_close > sell_line:
                if (today_close - sell_line) / sell_line > 0.005:
                    entry_price = sell_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                position = True
                trade_event = True  # Tradeereignis (direkter Einstieg)
                # Setze den Referenzwert für spätere Steuerberechnungen:
                last_entry_value = sim_value * (1 + day_return)
                sim_value = sim_value * (1 + day_return)
            elif today_close < buy_line:
                # Setze einen Trigger, dass ein Einstieg erwogen werden soll, sobald der Kurs wieder steigt.
                triggered_buy = True
                day_return = 0
                sim_value = sim_value
            else:
                day_return = 0
                sim_value = sim_value

    # === Intraday-Verletzungsabschlag (Risikoabschlag) ===
    # Wird sowohl im Long- als auch im Cash-Zustand (hier im simulierten Teil) angewandt.
    apply_penalty = False
    if position:
        if open_ > sell_line and today_close > sell_line:
            if low < sell_line * 0.995:
                apply_penalty = True
    else:
        if open_ < buy_line and today_close < buy_line:
            if high > buy_line * 1.005:
                apply_penalty = True

    if apply_penalty:
        # Der simulierte Teil erhält einen zusätzlichen Risikokorrekturfaktor.
        sim_value *= 0.985

    # --- Fester Portfolio-Teil: Tägliche Entwicklung mit fixen 0.0004 ---
    fixed_value = fixed_value * (1 + fixed_return)

    # --- Rebalancing beim Tradeereignis ---
    # Falls heute ein Trade (Einstieg oder Exit) im simulierten Teil stattgefunden hat,
    # wird das Gesamtportfolio (fester Teil + simulierte Strategie) zum Ende des Tages neu
    # auf 35 % (fixed) und 65 % (simuliert) aufgeteilt.
    if trade_event:
        total_value = fixed_value + sim_value
        fixed_value = total_value * 0.35
        sim_value = total_value * 0.65

    # Gesamtportfolio berechnen
    total_value = fixed_value + sim_value

    # Ergebnisse für den Tag speichern
    total_history.append(total_value)
    fixed_history.append(fixed_value)
    sim_history.append(sim_value)

# Ergebnisse in DataFrame eintragen und anzeigen
df["Total Portfolio"] = total_history
df["Fixed Part"] = fixed_history
df["Simulated Part"] = sim_history

# Beispiel-Ausgaben: Ersten und letzten Tag anzeigen
print("Erster Tag:")
print(df[["Date", "Price", "Total Portfolio", "Fixed Part", "Simulated Part"]].head(1))
print("\nLetzter Tag:")
print(df[["Date", "Price", "Total Portfolio", "Fixed Part", "Simulated Part"]].tail(1))


Erster Tag:
        Date    Price  Total Portfolio  Fixed Part  Simulated Part
0 2001-08-13  1670.96            100.0        35.0            65.0

Letzter Tag:
           Date    Price  Total Portfolio   Fixed Part  Simulated Part
4998 2021-06-21  8789.52      6708.727441  1201.555879     5507.171562


## Mischportfolio mit Bonuszertfikaten im Crash

In [209]:
import pandas as pd
import numpy as np

# ---------------------------
# 1. Daten einlesen & vorbereiten
# ---------------------------
df = pd.read_csv("data/S&P 500 TR Historical Data.csv",
                 delimiter=",",
                 parse_dates=["Date"],
                 dayfirst=False)

# Preis-Spalten bereinigen: Tausender-Trennzeichen entfernen und in float konvertieren
cols = ["Price", "Open", "High", "Low"]
for col in cols:
    df[col] = df[col].str.replace(",", "").astype(float)

# Reihenfolge so umkehren, dass das älteste Datum zuerst steht
df = df.iloc[::-1].reset_index(drop=True)

# ---------------------------
# 2. Indikatoren berechnen
# ---------------------------
# 200-Tage-SMA (Simple Moving Average)
df["SMA_200"] = df["Price"].rolling(window=200).mean()
# EMA kann zusätzlich berechnet werden, hier nutzen wir aber vorrangig SMA
df["EMA_200"] = df["Price"].ewm(span=200, adjust=False).mean()

# Für BuyLine und SellLine wird 99% des SMA_200 genommen (anpassbar)
df["SellLine"] = df["SMA_200"] * 0.99
df["BuyLine"]  = df["SMA_200"] * 0.99

# Tagesrendite (basierend auf Price)
df["Return"] = df["Price"].pct_change()

# ---------------------------
# 3. SIMULATIONSEINSTELLUNGEN & INITIALISIERUNG
# ---------------------------
initial_capital = 100

# Fixer Portfolio-Teil: 35 % des Kapitals, der täglich um 0.0004 (ca. 10%/250 Tage) wächst
fixed_return = 0.0004

# Simulierter Teil: 65 % des Kapitals
tax_rate = 0.05  # Steuer auf Gewinne (5 %) beim Exit aus der 3x-Position

# Anfangsallokation
fixed_value = initial_capital * 0.35
sim_value   = initial_capital * 0.65

# Bonus-Zertifikats-Komponenten:
# Seitwärtsrendite: 10% pro 250 Tage → 0.10/250 pro Tag
sideways_yield = 0.14 / 250

# Wir unterscheiden jetzt in drei Zustände für den simulierten Teil:
#  "3x"            → 3x gehebelte Position
#  "bonus_reverse" → Reverse Bonuszertifikat (initial, inverses Verhalten)
#  "bonus_long"    → Normales (long) Bonuszertifikat, dessen Rendite in S&P-Richtung verläuft

# Start: Nach einem Exit aus der 3x-Position (oder zu Beginn) halten wir ein reverse Bonuszertifikat.
mode = "bonus_reverse"

# Flag, falls ein Kauftrigger (für den Umstieg in 3x) aktiviert wurde.
triggered_buy = False

# Zum Tracking der steuerrelevanten Werte beim Einstieg in die 3x-Position
last_entry_value = None

# Zusätzlich speichern wir den S&P-Preis, zum Zeitpunkt des Kaufs des Bonuszertifikats.
# Wird benötigt, um den Drop von >15% zu messen.
# Bei Start setzen wir bonus_entry_spx auf den ersten verfügbaren Preis (z.B. am Tag 1)
bonus_entry_spx = df.iloc[1]["Price"]

# Listen zum Tracking der täglichen Portfolio-Werte
total_history = [fixed_value + sim_value]
fixed_history = [fixed_value]
sim_history   = [sim_value]

# ---------------------------
# 4. SIMULATIONSSCHLEIFE
# ---------------------------
for i in range(1, len(df)):
    # Tagesdaten
    prev_close = df.iloc[i - 1]["Price"]
    today_close = df.iloc[i]["Price"]
    open_ = df.iloc[i]["Open"]
    high  = df.iloc[i]["High"]
    low   = df.iloc[i]["Low"]
    today_ret = df.iloc[i]["Return"]

    # Schwellenwerte (Buy-/Sell-Line)
    buy_line  = df.iloc[i]["BuyLine"]
    sell_line = df.iloc[i]["SellLine"]

    trade_event = False  # Kennzeichnet, ob heute ein Trade (Ein-/Ausstieg) stattfindet
    new_sim_value = sim_value  # temporäre Variable für sim_value-Update

    # ---------------------------
    # A) 3x-Position: Direkter gehebelter Modus
    # ---------------------------
    if mode == "3x":
        # Exit-Signal: Wenn heutiger Schlusskurs unter der SellLine liegt
        if today_close < sell_line:
            if (sell_line - today_close) / sell_line > 0.005:
                exit_price = sell_line * 0.995
                ret = (exit_price - prev_close) / prev_close
            else:
                ret = (today_close - prev_close) / prev_close
            day_return = 3 * ret  # 3x gehebelter Tagesreturn
            new_sim_value = sim_value * (1 + day_return)

            # Steuerliche Anpassung beim Exit aus der 3x-Position
            if last_entry_value is not None:
                profit = (sim_value * (1 + day_return)) - last_entry_value
                if profit > 0:
                    tax_payment = tax_rate * profit
                    new_sim_value = sim_value * (1 + day_return) - tax_payment
                last_entry_value = None
            # Nach dem Exit gehen wir in den Bonuszertifikat-Modus (reverse)
            mode = "bonus_reverse"
            # Setze den Bonus-Kaufpreis (entry price) neu, da wir hier gerade umsteigen
            bonus_entry_spx = today_close
            trade_event = True
        else:
            # Fortsetzung der 3x-Position
            day_return = 3 * today_ret
            new_sim_value = sim_value * (1 + day_return)

    # ---------------------------
    # B) Bonuszertifikat-Modus (reverse oder long)
    # ---------------------------
    # Hier behandeln wir beide Varianten über mode.startswith("bonus")
    elif mode.startswith("bonus"):
        # Bestimme den Tagesreturn des Bonuszertifikats abhängig vom Typ:
        if mode == "bonus_reverse":
            bonus_return = sideways_yield - 0.5 * today_ret
        elif mode == "bonus_long":
            bonus_return = sideways_yield + 1 * today_ret
        else:
            bonus_return = 0  # Fallback (sollte nicht vorkommen)

        new_sim_value = sim_value * (1 + bonus_return)
        day_return = bonus_return

        # Zusätzliche Logik für reverse Bonuszertifikat:
        if mode == "bonus_reverse":
            # Wenn der aktuelle S&P-Schlusskurs mehr als 15% unter dem beim Kauf gespeicherten Preis liegt...
            if today_close < bonus_entry_spx * 0.7:
                # Umwandlung ins long Bonuszertifikat
                mode = "bonus_long"
                # Setze den Bonus-Kaufpreis neu (zum Zeitpunkt der Umwandlung)
                bonus_entry_spx = today_close

            # Andernfalls: Bei einem hohen Kurs (über SellLine) kann direkt in die 3x-Position gewechselt werden.
            # (Hier erfolgt der Exit aus dem reverse Zertifikat.)
            if today_close > sell_line:
                if (today_close - sell_line) / sell_line > 0.005:
                    entry_price = sell_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                mode = "3x"
                trade_event = True
                last_entry_value = new_sim_value * (1 + day_return)
                new_sim_value = new_sim_value * (1 + day_return)
            # Wenn der Kurs unter der BuyLine liegt, kann ein Kauf-Trigger gesetzt werden.
            elif today_close < buy_line:
                triggered_buy = True

        # Logik für long Bonuszertifikat:
        elif mode == "bonus_long":
            # Das long Zertifikat läuft mit positiver Korrelation, also:
            # Tagesreturn = sideways_yield + 1.2 * S&P Tagesreturn
            # Exit-Kriterium: Wenn der Schlusskurs die BuyLine überschreitet,
            # wechseln wir in die 3x-Position.
            if today_close > buy_line:
                if (today_close - buy_line) / buy_line > 0.005:
                    entry_price = buy_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    day_return = 3 * ret
                else:
                    day_return = 0
                mode = "3x"
                trade_event = True
                last_entry_value = new_sim_value * (1 + day_return)
                new_sim_value = new_sim_value * (1 + day_return)

    # ---------------------------
    # C) Intraday-Risikokorrektur (Penalty)
    # ---------------------------
    apply_penalty = False
    if mode == "3x":
        if open_ > sell_line and today_close > sell_line:
            if low < sell_line * 0.995:
                apply_penalty = True
    else:  # Im Bonus-Modus (egal ob reverse oder long)
        if open_ < buy_line and today_close < buy_line:
            if high > buy_line * 1.005:
                apply_penalty = True

    if apply_penalty:
        new_sim_value *= 0.985  # Risikoabschlag von 1.5%

    # ---------------------------
    # D) Fixed-Part Update
    # ---------------------------
    fixed_value = fixed_value * (1 + fixed_return)

    # ---------------------------
    # E) Rebalancing bei Tradeereignis
    # ---------------------------
    if trade_event:
        total_value = fixed_value + new_sim_value
        fixed_value = total_value * 0.35
        new_sim_value = total_value * 0.65

    # Gesamtportfolio-Wert aktualisieren
    sim_value = new_sim_value
    total_value = fixed_value + sim_value
    total_history.append(total_value)
    fixed_history.append(fixed_value)
    sim_history.append(sim_value)

# ---------------------------
# 5. Ergebnisse ins DataFrame eintragen & ausgeben
# ---------------------------
df["Total Portfolio"] = total_history
df["Fixed Part"] = fixed_history
df["Simulated Part"] = sim_history

print("Erster Tag:")
print(df[["Date", "Price", "Total Portfolio", "Fixed Part", "Simulated Part"]].head(1))
print("\nLetzter Tag:")
print(df[["Date", "Price", "Total Portfolio", "Fixed Part", "Simulated Part"]].tail(1))


Erster Tag:
        Date    Price  Total Portfolio  Fixed Part  Simulated Part
0 2001-08-13  1670.96            100.0        35.0            65.0

Letzter Tag:
           Date    Price  Total Portfolio   Fixed Part  Simulated Part
4998 2021-06-21  8789.52     10697.572353  1915.971556     8781.600796


In [215]:
import pandas as pd
import numpy as np

# === 1. SPX-Daten vorbereiten ===
spx_df = pd.read_csv("data/S&P 500 TR Historical Data.csv", delimiter=",", parse_dates=["Date"], dayfirst=False)
for col in ["Price", "Open", "High", "Low"]:
    spx_df[col] = spx_df[col].str.replace(",", "").astype(float)
spx_df = spx_df.iloc[::-1].reset_index(drop=True)
spx_df["SMA_200"] = spx_df["Price"].rolling(window=200).mean()
spx_df["EMA_200"] = spx_df["Price"].ewm(span=200, adjust=False).mean()
spx_df["SellLine"] = spx_df["SMA_200"] * 0.99
spx_df["BuyLine"] = spx_df["SMA_200"] * 0.99
spx_df["Return"] = spx_df["Price"].pct_change()

# === 2. Treasury Yield Daten einlesen und Bondpreise berechnen ===
treas_df = pd.read_csv("data/TreasuryYield.csv", parse_dates=["observation_date"])
treas_df = treas_df.rename(columns={"observation_date": "Date", "DGS10": "Yield"})
treas_df["Yield"] = pd.to_numeric(treas_df["Yield"], errors="coerce")
treas_df.dropna(inplace=True)
treas_df.sort_values("Date", inplace=True)

N = 100
coupon_rate = 0.03
T = 10
C = N * coupon_rate

def bond_price(yield_rate, C, N, T):
    r = yield_rate / 100
    return sum([C / (1 + r)**t for t in range(1, T+1)]) + N / (1 + r)**T

treas_df["BondPrice"] = treas_df["Yield"].apply(lambda r: bond_price(r, C, N, T))

# === 3. Merge SPX und Treasury Daten ===
spx_df.sort_values("Date", inplace=True)
spx_df.reset_index(drop=True, inplace=True)
merged_df = pd.merge_asof(spx_df, treas_df[["Date", "BondPrice"]], on="Date", direction="nearest")

# Fix für time-basierte Interpolation
merged_df.set_index("Date", inplace=True)
merged_df["BondPrice"] = merged_df["BondPrice"].interpolate(method="time")
merged_df.reset_index(inplace=True)

# Der restliche Code bleibt gleich wie oben und wird wieder verwendet.


# 4. Zur Simulation: Initialisierung der Portfolio-Größen
initial_capital = 100.0

# 65 % des Portfolios wird in die SPX-Strategie (3x gehebelte Position) investiert:
sim_value = initial_capital * 0.65
# 35 % des Portfolios werden in 3x gehebelte Treasuries investiert.
treasury_value = initial_capital * 0.35

# Parameter für die SPX-Strategie
position = False          # True, wenn aktuell in der 3x-SPX-Position
triggered_buy = False     # Signal: Wenn Price unter die BuyLine fiel, warten wir auf Rebound
last_entry_value = None   # Referenzwert beim letzten Einstieg (wichtig für steuerliche Berechnungen)
tax_rate = 0.05           # Steuer auf Gewinne des simulierten SPX-Teils (hier 5% – anpassbar)

# Für das Tracking der täglichen Werte
total_history = [initial_capital]
sim_history = [sim_value]
treasury_history = [treasury_value]

# Zur Berechnung der Treasury-Tagesrendite benötigen wir den BondPrice vom Vortag;
# daher speichern wir den heutigen BondPrice als Referenz
prev_bond_price = merged_df.iloc[0]["BondPrice"]

# 5. Simulation über alle Tage (beginnend ab Index 1)
for i in range(1, len(merged_df)):
    # --- SPX-Daten für diesen Tag ---
    prev_close = merged_df.iloc[i - 1]["Price"]
    today_close = merged_df.iloc[i]["Price"]
    open_ = merged_df.iloc[i]["Open"]
    high = merged_df.iloc[i]["High"]
    low = merged_df.iloc[i]["Low"]
    today_ret = merged_df.iloc[i]["Return"]
    buy_line = merged_df.iloc[i]["BuyLine"]
    sell_line = merged_df.iloc[i]["SellLine"]

    spx_day_return = 0.0
    exit_trade = False   # Kennzeichnet, dass heute im SPX-Teil ein Exit stattfindet (d.h. 3x Position wird geschlossen)
    trade_event = False  # Kennzeichnet einen Handelsvorgang (Einstieg oder Exit) im SPX-Teil

    # --- SPX-Strategielogik (3x gehebelte Strategie) ---
    if position:
        # Falls wir in der Position sind, prüfen wir, ob der Schlusskurs unter die SellLine fällt:
        if today_close < sell_line:
            # Bei starker Abweichung wird mit einem simulierten Exitpreis gearbeitet:
            if (sell_line - today_close) / sell_line > 0.005:
                exit_price = sell_line * 0.995
                # Hier wird der Return vom Vortagsschlusskurs zum Exitpreis berechnet
                ret = (exit_price - prev_close) / prev_close
            else:
                ret = (today_close - prev_close) / prev_close
            spx_day_return = 3 * ret
            position = False
            trade_event = True  # Es erfolgte ein Exit
            triggered_buy = (today_close < buy_line)  # Setze einen Trigger, falls Kurs unter BuyLine
            # --- Steuerliche Anpassung beim Exit ---
            if last_entry_value is not None:
                profit = sim_value * (1 + spx_day_return) - last_entry_value
                if profit > 0:
                    tax_payment = tax_rate * profit
                    # Simulierter SPX-Teil wird um die Steuer reduziert
                    sim_value = sim_value * (1 + spx_day_return) - tax_payment
                else:
                    sim_value = sim_value * (1 + spx_day_return)
                last_entry_value = None
                # Da sim_value bereits angepasst wurde, setzen wir spx_day_return auf 0, um doppelte Multiplikationen zu vermeiden.
                spx_day_return = 0
            else:
                sim_value = sim_value * (1 + spx_day_return)
        else:
            # Falls kein Exit-Signal vorliegt, läuft die 3x Position normal weiter:
            spx_day_return = 3 * today_ret
            sim_value = sim_value * (1 + spx_day_return)
    else:
        # Wir befinden uns im SPX-Cash-Teil (also noch nicht in der 3x Position)
        if triggered_buy:
            # Wurde ein Kauf-Trigger gesetzt, steigen wir ein, sobald der Kurs über der BuyLine liegt:
            if today_close > buy_line:
                if (today_close - buy_line) / buy_line > 0.005:
                    entry_price = buy_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    spx_day_return = 3 * ret
                else:
                    spx_day_return = 0
                position = True
                trade_event = True  # Handel (Einstieg)
                triggered_buy = False
                last_entry_value = sim_value * (1 + spx_day_return)
                sim_value = sim_value * (1 + spx_day_return)
            else:
                spx_day_return = 0
                # sim_value bleibt unverändert
        else:
            # Direkter Einstieg, falls der Schlusskurs über der SellLine liegt:
            if today_close > sell_line:
                if (today_close - sell_line) / sell_line > 0.005:
                    entry_price = sell_line * 1.005
                    ret = (today_close - entry_price) / entry_price
                    spx_day_return = 3 * ret
                else:
                    spx_day_return = 0
                position = True
                trade_event = True  # Handel (direkter Einstieg)
                last_entry_value = sim_value * (1 + spx_day_return)
                sim_value = sim_value * (1 + spx_day_return)
            elif today_close < buy_line:
                triggered_buy = True
                spx_day_return = 0
            else:
                spx_day_return = 0

    # --- Risikoabschlag (Intraday-Verletzungsabschlag) für den SPX-Teil ---
    apply_penalty = False
    if position:
        if open_ > sell_line and today_close > sell_line:
            if low < sell_line * 0.995:
                apply_penalty = True
    else:
        if open_ < buy_line and today_close < buy_line:
            if high > buy_line * 1.005:
                apply_penalty = True
    if apply_penalty:
        sim_value *= 0.985

    # --- Treasury-Teil (3x gehebelte Treasuries) ---
    # Berechne den Treasury-Tagesreturn aus den BondPrice-Werten
    today_bond_price = merged_df.iloc[i]["BondPrice"]
    treasury_return = (today_bond_price / prev_bond_price) - 1
    # 3× Hebel anwenden:
    treasury_value = treasury_value * (1 + 3 * treasury_return)
    # Setze für den nächsten Tag den heutigen BondPrice als Referenz:
    prev_bond_price = today_bond_price

    # --- Rebalancing beim Handelsereignis im SPX-Teil ---
    # Wird ein Trade (Kauf oder Verkauf) im SPX-Teil durchgeführt, erfolgt zum Tagesende eine Neuaufteilung.
    if trade_event:
        total_value = sim_value + treasury_value
        sim_value = total_value * 0.65
        treasury_value = total_value * 0.35

    # Gesamtportfolio berechnen
    total_value = sim_value + treasury_value

    # Portfolio-Werte speichern
    total_history.append(total_value)
    sim_history.append(sim_value)
    treasury_history.append(treasury_value)

# 6. Ergebnisse in das DataFrame eintragen und anzeigen
merged_df["Total Portfolio"] = total_history
merged_df["SPX Strategy Part"] = sim_history
merged_df["Treasury Part"] = treasury_history

# Beispiel-Ausgaben: Ersten und letzten Tag
print("Erster Tag:")
print(merged_df[["Date", "Price", "BondPrice", "Total Portfolio", "SPX Strategy Part", "Treasury Part"]].head(1))
print("\nLetzter Tag:")
print(merged_df[["Date", "Price", "BondPrice", "Total Portfolio", "SPX Strategy Part", "Treasury Part"]].tail(1))


Erster Tag:
        Date    Price  BondPrice  Total Portfolio  SPX Strategy Part  \
0 2001-08-13  1670.96  84.765996            100.0               65.0   

   Treasury Part  
0           35.0  

Letzter Tag:
           Date    Price   BondPrice  Total Portfolio  SPX Strategy Part  \
4998 2021-06-21  8789.52  113.833277      5571.779835        4819.266867   

      Treasury Part  
4998     752.512968  
